# Introduction to Matrix Product States (MPS) with NumPy

In the last section we looked at how to compress tensors using the TT-SVD algorithm which lead to a representation of the tensor in Tensor Train, or Matrix Product State (MPS) form. In this notebook we'll look at how to work with Matrix Product States (MPS) using NumPy. 

## Table of Contents:

1. Creating product states
2. Creating entangled states
3. Addition
4. Scalar multiplication
5. Getting elements of an MPS
6. Computing inner products
7. Computing expectation values
8. Exercises

### Utility Functions:

In [103]:
# Get bond dimensions/tt-ranks
def bdims(mps: list):
    return [site.shape[0] for site in mps] + [mps[-1].shape[-1]]

In [104]:
import numpy as np
from copy import copy

def restore_full(mps: list) -> np.ndarray:
    """
    Restore full tensor from an MPS/TT

    Args:
        mps: List of order-3 tensors representing an MPS/TT

    Return:
        The full tensor
    """
    tmp = copy(mps[0])
    dims = [site.shape[1] for site in mps]
    for site in mps[1:]:
        tmp = np.einsum('iuj,jvk->iuvk', tmp, site)
        tmp = tmp.reshape(tmp.shape[0], tmp.shape[1] * tmp.shape[2], tmp.shape[3])
    return tmp.reshape(dims)

## 1. Creating Product States

Product states are many-body quantum states that can be written as a products of states for each component of the system. For example $|\psi\rangle = |0\rangle \otimes |1\rangle \otimes |0\rangle \dots \otimes |0\rangle$. Product states can be represented by MPS with bond dimension 1.

In [105]:
def product_state_mps(state_list):
    """
    Generate an MPS representing a product state

    Args:
        state_list (list[list]]): A list of state vectors
    
    Returns:
        mps (list): List of MPS cores
    """
    return [np.array(s).reshape(1, len(s), 1) for s in state_list]

### Exercises
1. Create an MPS for the all-plus state: $|\psi\rangle = |++++++\rangle$, where $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$
2. Create an MPS for the ground state of the Hamiltonian $\hat{H}=\sigma_{y}\otimes\sigma_{y}\otimes\sigma_{y}\otimes\sigma_{y}$

## 2. Creating Entangled States

Entangled states are those that cannot be factorized into products. For these states, the bond dimension of the MPS must be greater than 1.

### Example: GHZ State

The GHZ state $|\text{GHZ}\rangle = \frac{1}{\sqrt{2}} \left( |00...0\rangle + |11...1\rangle \right)$ can be written with a bond dimension of 2.

In [106]:
def create_ghz_mps(N):
    """
    Create an MPS representation of the N-qubit GHZ state.
    
    Args:
        N (int): Number of qubits.
    
    Returns:
        mps (list): List of MPS cores
    """
    ghz_tensor_first = np.zeros((1, 2, 2))
    ghz_tensor_first[0, 0, 0] = 1 / np.sqrt(2)
    ghz_tensor_first[0, 1, 1] = 1 / np.sqrt(2)

    ghz_tensor_middle = np.zeros((2, 2, 2))
    ghz_tensor_middle[0, 0, 0] = 1
    ghz_tensor_middle[1, 1, 1] = 1
    
    ghz_tensor_last = np.zeros((2, 2, 1))
    ghz_tensor_last[0, 0, 0] = 1
    ghz_tensor_last[1, 1, 0] = 1
    
    # Construct the full MPS: [first tensor, middle tensors, last tensor]
    mps = [ghz_tensor_first] + [ghz_tensor_middle] * (N - 2) + [ghz_tensor_last]
    
    return mps

# Example: Create the MPS for a 5-qubit GHZ state
N = 5
ghz_mps = create_ghz_mps(N)

Let's check the bond dimensions

In [107]:
bdims(ghz_mps)

[1, 2, 2, 2, 2, 1]

Let's check this gives us the state we want by contracting the network to get the full tensor

In [108]:
full_tensor = restore_full(ghz_mps)

# Check the 2 coefficients that should by non-zero
print("Coefficient of |00000> = ", full_tensor[0, 0, 0, 0, 0])
print("Coefficient of |11111> = ", full_tensor[1, 1, 1, 1, 1])

# Check some other coefficients that should be zero
print("Coefficient of |01111> = ", full_tensor[0, 1, 1, 1, 1])
print("Coefficient of |01010> = ", full_tensor[0, 1, 0, 1, 0])
print("Coefficient of |00011> = ", full_tensor[0, 0, 0, 1, 1])

Coefficient of |00000> =  0.7071067811865475
Coefficient of |11111> =  0.7071067811865475
Coefficient of |01111> =  0.0
Coefficient of |01010> =  0.0
Coefficient of |00011> =  0.0


### Exercise
Create an MPS for the state $|\psi\rangle = \frac{1}{\sqrt{2}} \left( |000000\rangle - |111111\rangle \right)$

## 3. Adding MPS

All the basic operations that can be performed on vectors, such as adding, scalar multiplication, dot product etc, can also be performed with MPS. First we'll look at how to add two MPS.

Suppose you have two MPS representing quantum states $|\psi_1\rangle$ and $|\psi_2\rangle $, and you want to create an MPS for the state:

$
|\psi_{\text{sum}}\rangle = |\psi_1\rangle + |\psi_2\rangle
$

In order for the sum to make sense, the physical dimensions of the two MPSs must match.

Let the MPS representations of $|\psi_1\rangle$ and $|\psi_2\rangle$ be given by tensors $\{A^{[i]}_1\}$ and $\{A^{[i]}_2\}$ for sites $i = 1, \ldots, N $, where $N$ is the number of sites (qubits).

Then the MPS for $|\psi_{\text{sum}}\rangle$ will be given by the tensors:
$
\{
\begin{pmatrix}
A^{[1]}_1 & A^{[1]}_2
\end{pmatrix}
$,
$
\begin{pmatrix}
A^{[2]}_1 & 0 \\
0 & A^{[2]}_2
\end{pmatrix}
$, ...
$
\begin{pmatrix}
A^{[N-1]}_1 & 0 \\
0 & A^{[N-1]}_2
\end{pmatrix}
$,
$
\begin{pmatrix}
A^{[N]}_1 \\
A^{[N]}_2
\end{pmatrix}
\}
$

If $A^{[i]}_1$ has shape $(D_{i-1}^{(1)}, d_i, D_i^{(1)})$, where $d_i$ is the physical dimension, $D_{i-1}$ is the left bond dimension, and $D_i$ is the right bond dimension, and similarly for $A^{[i]}_2$, then $A^{[i]}_{\text{sum}}$ will have shape $(D_{i-1}^{(1)} + D_{i-1}^{(2)}, d_i, D_i^{(1)} + D_i^{(2)}) $.


### Exercise:

Confirm, by performing the matrix multiplications, that the above expression does indeed represent the sum of the MPS.


### Code for adding MPS

In [109]:
import numpy as np

def add_mps(mps1, mps2):
    """
    Add two MPS representations together.
    
    Args:
        mps1 (list): List of tensors representing the first MPS.
        mps2 (list): List of tensors representing the second MPS.
    
    Returns:
        mps_sum (list): List of tensors representing the MPS for the sum of the two states.
    """
    if len(mps1) != len(mps2):
        raise ValueError("The two MPS must have the same number of sites.")
    
    mps_sum = []
    
    for A1, A2 in zip(mps1, mps2):
        D1_left, d, D1_right = A1.shape
        D2_left, _, D2_right = A2.shape

        # Create a zero array with the correct dimensions
        A_sum = np.zeros((D1_left + D2_left, d, D1_right + D2_right))
        
        # Fill the upper-left block with A1
        A_sum[:D1_left, :, :D1_right] = A1
        
        # Fill the lower-right block with A2
        A_sum[D1_left:, :, D1_right:] = A2
        
        # Append the resulting tensor to the new MPS list
        mps_sum.append(A_sum)

    # Sum over boundary bond dimensions
    mps_sum[0] = np.sum(mps_sum[0], axis=0, keepdims=True)
    mps_sum[-1] = np.sum(mps_sum[-1], axis=-1, keepdims=True)
    
    return mps_sum

### Bond Dimension Growth
The bond dimension of the resulting MPS is larger than that of the starting MPS. If we're not carefull when adding multiple MPS together, the bond dimension can get out of hand. However it is often possible to compress the MPS resulting from an addition to one with a smaller bond dimension.

### Example

Let's try to construct an MPS for the GHZ state we looked at above but this time using the add function.

In [110]:
mps_00000 = product_state_mps([[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]])
mps_11111 = product_state_mps([[0, 1], [0, 1], [0, 1], [0, 1], [0, 1]])

In [111]:
ghz_mps_add = add_mps(mps_00000, mps_11111)

Check the bond dimensions:

In [112]:
bdims(ghz_mps_add)

[1, 2, 2, 2, 2, 1]

Check the coefficients

In [113]:
full_tensor = restore_full(ghz_mps_add)

# Check the 2 coefficients that should by non-zero
print("Coefficient of |00000> = ", full_tensor[0, 0, 0, 0, 0])
print("Coefficient of |11111> = ", full_tensor[1, 1, 1, 1, 1])

# Check some other coefficients that should be zero
print("Coefficient of |01111> = ", full_tensor[0, 1, 1, 1, 1])
print("Coefficient of |01010> = ", full_tensor[0, 1, 0, 1, 0])
print("Coefficient of |00011> = ", full_tensor[0, 0, 0, 1, 1])

Coefficient of |00000> =  1.0
Coefficient of |11111> =  1.0
Coefficient of |01111> =  0.0
Coefficient of |01010> =  0.0
Coefficient of |00011> =  0.0


As we can see, the structure of the state is correct but we are missing the normalisation factor of $1/\sqrt{2}$. To properly normalise the state we need to know how to multiply an MPS by a scalar.

## 4. Scalar Multiplication

Multiplying an MPS by a scalar is very straightforward; we just need to multiply one of the tensors by the scalar.

### Code for multiplying MPS by a scalar

In [114]:
def scale_mps(mps, scalar):
    """
        Multiplies an MPS by a scalar, changes the MPS in place.
        
        Args:
            mps (list): List of tensors representing the MPS.
            scalar (float): The scalar
    """
    
    mps[0] = mps[0] * scalar
    return mps

With this, we can now normalise the GHZ state we created above:

In [115]:
scale_mps(ghz_mps_add, 1 / np.sqrt(2))

full_tensor = restore_full(ghz_mps_add)
# Check the 2 coefficients that should by non-zero
print("Coefficient of |00000> = ", full_tensor[0, 0, 0, 0, 0])
print("Coefficient of |11111> = ", full_tensor[1, 1, 1, 1, 1])

Coefficient of |00000> =  0.7071067811865475
Coefficient of |11111> =  0.7071067811865475


### Exercises

1. Using the functions defined above, make an MPS for the state $|\psi \rangle = \frac{1}{\sqrt{3}}(|010101\rangle-|101010\rangle+|111000\rangle$.
2. Check that the MPS is correct by restoring the full tensor.
3. Create an MPS for the function $\sin(\omega x)$ where $x$ is defined on a unit grid between 0 and 1267650600228229401496703205375.
4. Confirm your answer to 3 by evaluating several tensor elements.

## 5. Getting Tensor Elements

So far we have been checking our MPS by reconstructing the full tensor, however remember that the point of working with MPS is to be able to treat tensors that are far too large to be written otherwise than in decomposed form. Luckily, we can compute tensor elements of an MPS without contracting to form the full tensor - we just need to select the indices on the tensor cores *before* contracting.

### Code for computing tensor elements:

In [116]:
from functools import reduce
def get_index(mps: list, inds):
    """
    Get element of an MPS for a set of indices.

    Args:
        mps (list): List of tensors representing the MPS.
        inds (list): The list of indices
    """
    return reduce(np.matmul, (site[:, ind, :] for site, ind in zip(mps, inds)))[0,0]

Now we can compute tensor elements without recontructing the full tensor, let's use this to check our GHZ state from before:

In [117]:
# Check the 2 coefficients that should by non-zero
print("Coefficient of |00000> = ", get_index(ghz_mps, [0, 0, 0, 0, 0]))
print("Coefficient of |11111> = ", get_index(ghz_mps, [1, 1, 1, 1, 1]))

# Check some other coefficients that should be zero
print("Coefficient of |01111> = ", get_index(ghz_mps, [0, 1, 1, 1, 1]))
print("Coefficient of |01010> = ", get_index(ghz_mps, [0, 1, 0, 1, 0]))
print("Coefficient of |00011> = ", get_index(ghz_mps, [0, 0, 0, 1, 1]))

Coefficient of |00000> =  0.7071067811865475
Coefficient of |11111> =  0.7071067811865475
Coefficient of |01111> =  0.0
Coefficient of |01010> =  0.0
Coefficient of |00011> =  0.0


We can see that the results are the same as before.

## 6. Computing Inner Products

The inner product between two MPS $|\psi_1\rangle$ and $|\psi_2\rangle$ is given by $\langle\psi_1|\psi_2\rangle$. The inner product is a measure of the *overlap* between two MPS.

To compute the inner product efficiently, we must contract the tensors iteratively along the chain.

![](../img/mps_inner.png)

In [118]:
def inner_product(mps1, mps2):
    """
    Compute the inner product <psi1|psi2> between two MPS.
    
    Args:
        mps1 (list): List of tensors representing the first MPS.
        mps2 (list): List of tensors representing the second MPS.
    
    Returns:
        float: The inner product <psi1|psi2>.
    """
    
    contraction_result = np.ones((1, 1))
    for A1, A2 in zip(mps1, mps2):
        # Contract along the physical dimension
        #   /-i-(A2)-l-  
        #  [c]   |k   
        #   \-j-(A1)-m- 
        contraction_result = np.einsum('ij,jkm,ikl->lm', contraction_result, np.conj(A1), A2)
    return contraction_result[0, 0]

### Examples

We can use the inner product to compute the norm of an MPS: $|\psi| =\sqrt{\langle\psi|\psi\rangle}$

In [119]:
norm = np.sqrt(inner_product(mps_ghz, mps_ghz))
print("Norm of GHZ state: ", norm)

Norm of GHZ state:  1.0


Or the overlap between two different states:

In [122]:
mps_00000 = product_state_mps([[1, 0], [1, 0], [1, 0], [1, 0], [1, 0]])
print("Overlap of GHZ with |00000>: ", inner_product(ghz_mps, mps_00000))

Overlap of GHZ with |00000>:  0.7071067811865475


### Exercises

1. What is the compexity of computing the inner product of two MPS?
2. Compute the inner product of a 6 qubit GHZ state with the all-plus state: $|\psi\rangle = |++++++\rangle$

## 7. Computing Expectation Values

Computing expectation values is very similar to computing inner products. We will compute the expectation value of a local operator $O$ with respect to a Matrix Product State (MPS) $|\psi\rangle$:
$
\langle \psi | O | \psi \rangle
$

![](../img/mps_exp.png)

In [124]:
def expectation_value(mps, operator, site):
    """
    Compute the expectation value <psi|O|psi> for a local operator at a specified site.
    
    Args:
        mps (list): List of tensors representing the MPS.
        operator (np.ndarray): Local operator acting on the physical space (shape (d, d)).
        site (int): The index of the site where the operator is applied.
    
    Returns:
        float: The expectation value <psi|O|psi>.
    """

    contraction_result = np.ones((1, 1))
    for i, A in enumerate(mps):
        if i == site:
            # Insert the operator at the specified site
            #   /-i-(A)-l-  
            #   |    |t
            #  [c]  [O]   
            #   |    |s
            #   \-j-(A)-m- 
            contraction_result = np.einsum('ij,jsm,itl,st->lm', contraction_result, np.conj(A), A, operator)
        else:
            # Regular tensor contraction without the operator
            #   /-i-(A)-l-  
            #  [c]   |k   
            #   \-j-(A)-m- 
            contraction_result = np.einsum('ij,jkm,ikl->lm', contraction_result, np.conj(A), A)
    
    # Return the final scalar expectation value
    return contraction_result[0, 0]

## Example

Compute the expectation value of the Pauli spin operators on the GHZ state:

In [135]:
# Define the Pauli spin operators
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, -1]])

print("Expectation of sx on qubit 3: ", expectation_value(ghz_mps, sx, 3))
print("Expectation of sy on qubit 3: ", expectation_value(ghz_mps, sy, 3))
print("Expectation of sz on qubit 3: ", expectation_value(ghz_mps, sz, 3))

Expectation of sx on qubit 3:  0.0
Expectation of sy on qubit 3:  0j
Expectation of sz on qubit 3:  0.0


### Exercise

Confirm the above results for the expectation values of the Pauli operators on the GHZ state with pen and paper calculations.